# Develop composition analyses.

Here we guide the development of composition analyses.
The code is developed as a class. Here we document and test the development.

In [1]:
import os
import numpy as np
import pandas as pd
project =\
    os.path.join(
        "/gpfs/bbp.cscs.ch/project")
proj_sscx_diss=\
    os.path.join(
        project,
        "proj83")
data_sscx_diss=\
    os.path.join(
        proj_sscx_diss,
        "data")
atlases_sscx_diss=\
    os.path.join(
        data_sscx_diss,
        "atlas/S1/MEAN")
circuits_sscx_diss=\
    os.path.join(
        proj_sscx_diss,
        "circuits")

In [2]:
from neuro_dmt.models.bluebrain.circuit.atlas import\
    BlueBrainCircuitAtlas
from neuro_dmt.models.bluebrain.circuit.model import\
    BlueBrainCircuitModel,\
    CircuitProvenance
from neuro_dmt.models.bluebrain.circuit.adapter import\
    BlueBrainCircuitAdapter
atlas_bio_m =\
    BlueBrainCircuitAtlas(
        path=os.path.join(
                atlases_sscx_diss, "P14-MEAN"))
path_bio_m =\
    os.path.join(
        circuits_sscx_diss,
        "Bio_M/20191206")
circuit_bio_m =\
    BlueBrainCircuitModel(
        path_circuit_data=path_bio_m,
            provenance=CircuitProvenance(
                label="SSCxRatDisseminationBioM",
                authors=["BBP Team"],
                release_date="20191212",
                location=path_bio_m,
                animal="Wistar Rat",
                age="P14",
                brain_region="SSCx"))

('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'roi', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'roi', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'roi', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mtype', 'etype', 'synapse_class')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'roi', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'roi')


In [ ]:
from neuro_dmt.models.bluebrain.circuit.mock.test.mock_circuit_light import\
    circuit_composition,\
    circuit_connectivity
from neuro_dmt.models.bluebrain.circuit.mock.circuit\
    import MockBlueBrainCircuitModel
mock_circuit_model = MockBlueBrainCircuitModel(
    circuit_composition,
    circuit_connectivity)

In [3]:
adapter = BlueBrainCircuitAdapter()

In [4]:
from neuro_dmt.data import rat
from neuro_dmt.library.users.visood.sscx_dissemination.analyses.composition import\
    BrainCircuitCompositionAnalysis

composition_analysis =\
    BrainCircuitCompositionAnalysis(
        phenomenon="circuit_composition")


In [22]:
circuit_bio_m.atlas.region_layer.get_mask(
    region="S1HL",
    layer="L1")

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, Fal

In [8]:
mtypes = adapter.get_mtypes(circuit_bio_m)
regions = adapter.get_brain_regions(circuit_bio_m)
layers = adapter.get_layers(circuit_bio_m)
measurement_parameters =\
    pd.DataFrame({
        "mtype":  [m for m in mtypes for _ in regions for _ in layers],
        "region": [r for _ in mtypes for r in regions for _ in layers],
        "layer":  [l for _ in mtypes for _ in regions for l in layers]})

mtype_cell_density =\
    composition_analysis.get_cell_density_measurement(
        circuit_bio_m,
        adapter,
        measurement_parameters)

VoxcellError: ("Region not found: 'region'", 'occurred at index 0')

In [16]:
terminology.circuit.get_spatial_query(measurement_parameters.iloc[0].to)

TypeError: pop() takes 2 positional arguments but 3 were given

In [18]:
from neuro_dmt import terminology
terminology.circuit.get_spatial_query(measurement_parameters.iloc[0].to_dict())

{'region': 'S1FL',
 'layer': 'L1',
 'depth': None,
 'height': None,
 'mesocolumn': None,
 'hypercolumn': None}

In [5]:
circuit_bio_m.brain_regions
#adapter.get_brain_regions(circuit_bio_m)

[S1FL, S1Sh, S1HL, S1Tr]
Categories (4, object): [S1FL, S1Sh, S1HL, S1Tr]

In [ ]:
mtypes = adapter.get_mtypes(mock_circuit_model)
regions = adapter.get_brain_regions(mock_circuit_model)
layers = adapter.get_layers(mock_circuit_model)
measurement_parameters =\
    pd.DataFrame({
        "mtype":  [m for m in mtypes for _ in regions for _ in layers],
        "region": [r for _ in mtypes for r in regions for _ in layers],
        "layer":  [l for _ in mtypes for _ in regions for l in layers]})

mock_mtype_cell_density =\
    composition_analysis.get_cell_density_measurement(
        mock_circuit_model,
        adapter,
        measurement_parameters)

In [6]:
import pandas as pd
from neuro_dmt.models.bluebrain.circuit.geometry import Cuboid

regions = adapter.get_brain_regions(mock_circuit_model)
layers = adapter.get_layers(mock_circuit_model)
regions_and_layers = pd.DataFrame({
    "region": [r for r in regions for _ in layers],
    "layer":  [l for _ in regions for l in layers]
})
def _get_cell_density(**spatial_query):
    position =\
        adapter.random_position(mock_circuit_model, **spatial_query)
    cuboid =\
        Cuboid(position - 25., position + 25.)
    return\
        adapter.get_cells(mock_circuit_model, roi=cuboid.bbox).shape[0] / cuboid.volume
            
    
mp = pd.concat(
    [regions_and_layers for _ in range(2)]
)
m = mp.assign(
    cell_density=lambda df: df.apply(
        lambda row: _get_cell_density(**row),
        axis=1),
    dataset=adapter.get_label(mock_circuit_model)
).set_index(
    ["dataset"] + list(mp.columns.values))

NameError: name 'mock_circuit_model' is not defined

In [ ]:
from neuro_dmt.data import rat
rat.defelipe2017.citation

In [ ]:
mock_circuit_model.provenance.field_dict

In [ ]:
from neuro_dmt.models.bluebrain.circuit.geometry import Cuboid
from neuro_dmt import terminology

position =\
    adapter.random_position(
        mock_circuit_model,
        region="S1HL",
        layer="L2")
cuboid =\
    Cuboid(
        position - 50.,
        position + 50.)
cells =\
    adapter.get_cells(
        mock_circuit_model, 
#        properties=[terminology.cell.synapse_class, terminology.cell.mtype],
        roi=cuboid.bbox,
        mtype="L23_MC")
number_synapse_class =\
    cells.groupby(terminology.cell.synapse_class).agg("size")

In [ ]:
cells